In [2]:
pip install torch==2.2.2 torchvision==0.17.2 torchaudio==2.2.2

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
#Load libraries
import os
import numpy as np
import torch
import glob
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torchvision
import pathlib

In [4]:
#checking for device
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
print(device)

cpu


In [6]:
#Transforms
transformer=transforms.Compose([
    transforms.Resize((150,150)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),  #0-255 to 0-1, numpy to tensors
    transforms.Normalize([0.5,0.5,0.5], # 0-1 to [-1,1] , formula (x-mean)/std
                        [0.5,0.5,0.5])
])

In [7]:
from torch.utils.data import DataLoader


In [9]:
#Dataloader

#Path for training and testing directory
train_path='C:\\Users\\LENOVO\\OneDrive\\Desktop\\computer vision\\training'
test_path='C:\\Users\\LENOVO\\OneDrive\\Desktop\\computer vision\\testing'

train_loader=DataLoader(
    torchvision.datasets.ImageFolder(train_path,transform=transformer),
    batch_size=32, shuffle=True
)
test_loader=DataLoader(
    torchvision.datasets.ImageFolder(test_path,transform=transformer),
    batch_size=32, shuffle=True
)

In [10]:
#categories
root=pathlib.Path(train_path)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])

In [11]:
print(classes)

['100%', '30%', '60%', '90%']


In [12]:
#CNN Network


class ConvNet(nn.Module):
    def __init__(self,num_classes=4):
        super(ConvNet,self).__init__()
        
        #Output size after convolution filter
        #((w-f+2P)/s) +1
        
        #Input shape= (32,3,150,150)
        
        self.conv1=nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1)
        #Shape= (20,12,150,150)
        self.bn1=nn.BatchNorm2d(num_features=12)
        #Shape= (20,12,150,150)
        self.relu1=nn.ReLU()
        #Shape= (20,12,150,150)
        
        self.pool=nn.MaxPool2d(kernel_size=2)
        #Reduce the image size be factor 2
        #Shape= (20,12,75,75)
        
        
        self.conv2=nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
        #Shape= (20,20,75,75)
        self.relu2=nn.ReLU()
        #Shape= (20,20,75,75)
        
        
        
        self.conv3=nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
        #Shape= (20,32,75,75)
        self.bn3=nn.BatchNorm2d(num_features=32)
        #Shape= (20,32,75,75)
        self.relu3=nn.ReLU()
        #Shape= (20,32,75,75)
        
        
        self.fc=nn.Linear(in_features=75 * 75 * 32,out_features=num_classes)
        
        
        
        #Feed forwad function
        
    def forward(self,input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)
            
        output=self.pool(output)
            
        output=self.conv2(output)
        output=self.relu2(output)
            
        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output)
            
            
            #Above output will be in matrix form, with shape (20,32,75,75)
            
        output=output.view(-1,32*75*75)
            
            
        output=self.fc(output)
            
        return output
            
        

In [13]:
model=ConvNet(num_classes=4).to(device)

In [14]:
#Optmizer and loss function
optimizer=Adam(model.parameters(),lr=0.001,weight_decay=0.0001)
loss_function=nn.CrossEntropyLoss()

In [15]:
num_epochs=10

In [16]:
#calculating the size of training and testing images
train_count=len(glob.glob(train_path+'/**/*.jpg'))
test_count=len(glob.glob(test_path+'/**/*.jpg'))

In [17]:
print(train_count,test_count)

443 443


In [18]:
#Model training and saving best model

best_accuracy=0.0

for epoch in range(num_epochs):
    
    #Evaluation and training on training dataset
    model.train()
    train_accuracy=0.0
    train_loss=0.0
    
    for i, (images,labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        optimizer.zero_grad()
        
        outputs=model(images)
        loss=loss_function(outputs,labels)
        loss.backward()
        optimizer.step()
        
        
        train_loss+= loss.cpu().data*images.size(0)
        _,prediction=torch.max(outputs.data,1)
        
        train_accuracy+=int(torch.sum(prediction==labels.data))
        
    train_accuracy=train_accuracy/train_count
    train_loss=train_loss/train_count
    
    
    # Evaluation on testing dataset
    model.eval()
    
    test_accuracy=0.0
    for i, (images,labels) in enumerate(test_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        outputs=model(images)
        _,prediction=torch.max(outputs.data,1)
        test_accuracy+=int(torch.sum(prediction==labels.data))
    
    test_accuracy=test_accuracy/test_count
    
    
    print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy)+' Test Accuracy: '+str(test_accuracy))
    
    #Save the best model
    if test_accuracy>best_accuracy:
        torch.save(model.state_dict(),'best_checkpoint.model')
        best_accuracy=test_accuracy

C:\Users\LENOVO\AppData\Roaming\Python\Python312\site-packages\PIL\Image.py:1000: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch: 0 Train Loss: tensor(19.2168) Train Accuracy: 0.5079006772009029 Test Accuracy: 0.22799097065462753
Epoch: 1 Train Loss: tensor(4.7786) Train Accuracy: 0.7381489841986456 Test Accuracy: 0.35665914221218964
Epoch: 2 Train Loss: tensor(2.3151) Train Accuracy: 0.835214446952596 Test Accuracy: 0.8939051918735892
Epoch: 3 Train Loss: tensor(1.5420) Train Accuracy: 0.9345372460496614 Test Accuracy: 0.909706546275395
Epoch: 4 Train Loss: tensor(0.8673) Train Accuracy: 0.9729119638826185 Test Accuracy: 0.9796839729119639
Epoch: 5 Train Loss: tensor(0.7702) Train Accuracy: 0.9706546275395034 Test Accuracy: 0.945823927765237
Epoch: 6 Train Loss: tensor(0.7211) Train Accuracy: 0.9751693002257337 Test Accuracy: 0.9887133182844243
Epoch: 7 Train Loss: tensor(1.8121) Train Accuracy: 0.9435665914221218 Test Accuracy: 0.9142212189616253
Epoch: 8 Train Loss: tensor(0.8836) Train Accuracy: 0.9932279909706546 Test Accuracy: 1.036117381489842
Epoch: 9 Train Loss: tensor(0.5768) Train Accuracy: 1.01